Установка и загрузка необходимых библиотек:

In [ ]:
using Pkg

Pkg.add("ResumableFunctions")
Pkg.add("GAP")

In [1]:
using ResumableFunctions
using GAP

 ┌───────┐   GAP 4.12.2 of 2022-12-18
 │  GAP  │   https://www.gap-system.org
 └───────┘   Architecture: x86_64-pc-linux-gnu-julia1.9-64-kv8
 Configuration:  gmp 6.2.1, Julia GC, Julia 1.9.1
 Loading the library and packages ...
 Packages:   AClib 1.3.2, Alnuth 3.2.1, AtlasRep 2.1.6, AutPGrp 1.11, 
             CRISP 1.4.6, Cryst 4.1.25, CrystCat 1.1.10, CTblLib 1.3.4, 
             FactInt 1.6.3, FGA 1.4.0, GAPDoc 1.6.6, IRREDSOL 1.4.4, 
             JuliaInterface 0.9.6, LAGUNA 3.9.5, Polenta 1.3.10, 
             Polycyclic 2.16, PrimGrp 3.4.3, RadiRoot 2.9, ResClasses 4.7.3, 
             SmallGrp 1.5.1, Sophus 1.27, SpinSym 1.5.2, TomLib 1.2.9, 
             TransGrp 3.6.3, utils 0.81
 Try '??help' for help. See also '?copyright', '?cite' and '?authors'


In [ ]:
Определение необходимых функций:

In [2]:
const s_char = ["G.1", "G.2"]
const char = ["*G.1", "*G.2", "*G.1^-1", "*G.2^-1"]

const transitions = 
    [1 2 4; 
     2 3 1; 
     3 4 2;
     4 1 3]

4×3 Matrix{Int64}:
 1  2  4
 2  3  1
 3  4  2
 4  1  3

In [3]:
struct Relator
    word::String
    num_a::Int64
    num_b::Int64
end


function relators_enc(length::Int64)
    Iterators.product(1:2, ntuple(i->1:3, length - 1)...)
end


@resumable function relators(length::Int64)
    for enc in relators_enc(length)
        (ind, cons...) = enc
        t = s_char[ind]
        
        num_a = ind == 1 ? 1 : 0
        num_b = ind == 2 ? 1 : 0
            
        for i in cons
            ind = transitions[ind, i] 
            t *= char[ind]
            
            if ind == 1
                num_a += 1
            elseif ind == 2
                num_b += 1
            elseif ind == 3
                num_a -= 1
            else
                num_b -= 1
            end
        end

        @yield Relator(
            t,
            num_a,
            num_b,
        )
    end
end


@resumable function perfect_presentations(min_len::Int64, max_len::Int64)
    for i = min_len : max_len
        for j = min_len : i
            for r_0 in relators(i)
                for r_1 in relators(j)
                    det = r_0.num_a * r_1.num_b - r_0.num_b * r_1.num_a
                    if det == 1 || det == -1
                        @yield (r_0.word, r_1.word)
                    end
                end
            end
        end
    end
end

perfect_presentations (generic function with 1 method)

In [ ]:
Генерация копредставлений:

In [4]:
GAP.Globals.CosetTableDefaultMaxLimit = 500

result = []

function main()
    GAP.Globals.G = @gap "FreeGroup(\"a\", \"b\")"
    
    for (w_1, w_2) in perfect_presentations(2, 7)
        T = GAP.evalstr("G / [$w_1, $w_2]")
        
        order = try 
            GAP.Globals.Order(T)
        catch _
            Nothing
        end

        if order != Nothing && order != 1
            push!(result, (w_1, w_2, order))
            println((w_1, w_2, order))
        end
    end
end

main()

("G.1*G.2^-1*G.1*G.2*G.2*G.2*G.2", "G.2*G.1^-1*G.2*G.1*G.1", 120)
("G.1*G.2^-1*G.1*G.2*G.2*G.2*G.2", "G.1*G.2^-1*G.1^-1*G.1^-1*G.2^-1", 120)
("G.1*G.2^-1*G.1*G.2*G.2*G.2*G.2", "G.2*G.1*G.1*G.2*G.1^-1", 120)
("G.1*G.2^-1*G.1*G.2*G.2*G.2*G.2", "G.1*G.1*G.2*G.1^-1*G.2", 120)
("G.1*G.2^-1*G.1*G.2*G.2*G.2*G.2", "G.1*G.2*G.1^-1*G.2*G.1", 120)
("G.2*G.1*G.2*G.1^-1*G.1^-1*G.1^-1*G.1^-1", "G.1*G.2*G.1*G.2^-1*G.2^-1", 120)
("G.2*G.1*G.2*G.1^-1*G.1^-1*G.1^-1*G.1^-1", "G.2*G.1*G.2^-1*G.2^-1*G.1", 120)
("G.2*G.1*G.2*G.1^-1*G.1^-1*G.1^-1*G.1^-1", "G.1*G.2^-1*G.2^-1*G.1*G.2", 120)
("G.2*G.1*G.2*G.1^-1*G.1^-1*G.1^-1*G.1^-1", "G.2*G.2*G.1^-1*G.2^-1*G.1^-1", 120)
("G.2*G.1*G.2*G.1^-1*G.1^-1*G.1^-1*G.1^-1", "G.2*G.1^-1*G.2^-1*G.1^-1*G.2", 120)
("G.1*G.2*G.1*G.2^-1*G.2^-1*G.2^-1*G.2^-1", "G.2*G.1*G.2*G.1^-1*G.1^-1", 120)
("G.1*G.2*G.1*G.2^-1*G.2^-1*G.2^-1*G.2^-1", "G.1*G.2^-1*G.1^-1*G.2^-1*G.1", 120)
("G.1*G.2*G.1*G.2^-1*G.2^-1*G.2^-1*G.2^-1", "G.1*G.1*G.2^-1*G.1^-1*G.2^-1", 120)
("G.1*G.2*G.1*G.2^-1*G.2^

("G.1*G.2*G.2*G.2*G.2*G.1*G.2^-1", "G.2*G.1^-1*G.2*G.1*G.1", 120)
("G.1*G.2*G.2*G.2*G.2*G.1*G.2^-1", "G.1*G.2^-1*G.1^-1*G.1^-1*G.2^-1", 120)
("G.1*G.2*G.2*G.2*G.2*G.1*G.2^-1", "G.2*G.1*G.1*G.2*G.1^-1", 120)
("G.1*G.2*G.2*G.2*G.2*G.1*G.2^-1", "G.1*G.1*G.2*G.1^-1*G.2", 120)
("G.1*G.2*G.2*G.2*G.2*G.1*G.2^-1", "G.1*G.2*G.1^-1*G.2*G.1", 120)
("G.2*G.1^-1*G.1^-1*G.1^-1*G.1^-1*G.2*G.1", "G.1*G.2*G.1*G.2^-1*G.2^-1", 120)
("G.2*G.1^-1*G.1^-1*G.1^-1*G.1^-1*G.2*G.1", "G.2*G.1*G.2^-1*G.2^-1*G.1", 120)
("G.2*G.1^-1*G.1^-1*G.1^-1*G.1^-1*G.2*G.1", "G.1*G.2^-1*G.2^-1*G.1*G.2", 120)
("G.2*G.1^-1*G.1^-1*G.1^-1*G.1^-1*G.2*G.1", "G.2*G.2*G.1^-1*G.2^-1*G.1^-1", 120)
("G.2*G.1^-1*G.1^-1*G.1^-1*G.1^-1*G.2*G.1", "G.2*G.1^-1*G.2^-1*G.1^-1*G.2", 120)
("G.1*G.1*G.1*G.2*G.1^-1*G.2*G.1", "G.1*G.2^-1*G.1*G.2*G.2", 120)
("G.1*G.1*G.1*G.2*G.1^-1*G.2*G.1", "G.2*G.1*G.2^-1*G.1*G.2", 120)
("G.1*G.1*G.1*G.2*G.1^-1*G.2*G.1", "G.2*G.2*G.1*G.2^-1*G.1", 120)
("G.1*G.1*G.1*G.2*G.1^-1*G.2*G.1", "G.2*G.1^-1*G.2^-1*G.2^-1*G.1^-1

("G.2*G.1^-1*G.1^-1*G.2*G.1*G.1*G.1", "G.2*G.1*G.2^-1*G.1*G.2*G.1^-1", 120)
("G.2*G.1^-1*G.1^-1*G.2*G.1*G.1*G.1", "G.1*G.2^-1*G.1^-1*G.2*G.1^-1*G.2^-1", 120)
("G.2*G.1^-1*G.1^-1*G.2*G.1*G.1*G.1", "G.2*G.1^-1*G.2*G.1*G.2^-1*G.1", 120)
("G.2*G.1^-1*G.1^-1*G.2*G.1*G.1*G.1", "G.1*G.2^-1*G.1*G.2*G.1^-1*G.2", 120)
("G.2*G.1^-1*G.1^-1*G.2*G.1*G.1*G.1", "G.2*G.1^-1*G.2^-1*G.1*G.2^-1*G.1^-1", 120)
("G.2*G.1^-1*G.1^-1*G.2*G.1*G.1*G.1", "G.1*G.2*G.1^-1*G.2*G.1*G.2^-1", 120)
("G.1*G.1*G.1*G.1*G.1*G.2*G.2", "G.1*G.2*G.1*G.2^-1*G.1*G.2", 120)
("G.1*G.1*G.1*G.1*G.1*G.2*G.2", "G.1*G.2^-1*G.1*G.2*G.1*G.2", 120)
("G.1*G.1*G.1*G.1*G.1*G.2*G.2", "G.2*G.1*G.2*G.1*G.2^-1*G.1", 120)
("G.1*G.1*G.1*G.1*G.1*G.2*G.2", "G.2*G.1*G.2^-1*G.1*G.2*G.1", 120)
("G.1*G.1*G.1*G.1*G.1*G.2*G.2", "G.2*G.1^-1*G.2^-1*G.1^-1*G.2^-1*G.1^-1", 120)
("G.1*G.1*G.1*G.1*G.1*G.2*G.2", "G.1*G.2*G.1*G.2*G.1*G.2^-1", 120)
("G.2*G.2*G.2*G.2*G.2*G.1^-1*G.1^-1", "G.2*G.1^-1*G.2*G.1*G.2*G.1^-1", 120)
("G.2*G.2*G.2*G.2*G.2*G.1^-1*G.1^-1", "G.2

("G.2*G.1^-1*G.2^-1*G.2^-1*G.1^-1*G.2*G.2", "G.2*G.1*G.2^-1*G.1^-1*G.2^-1*G.1", 120)
("G.2*G.1^-1*G.2^-1*G.2^-1*G.1^-1*G.2*G.2", "G.1*G.2*G.1*G.2^-1*G.1^-1*G.2^-1", 120)
("G.2*G.1^-1*G.2^-1*G.2^-1*G.1^-1*G.2*G.2", "G.2*G.1*G.2*G.1^-1*G.2^-1*G.1^-1", 120)
("G.2*G.1^-1*G.2^-1*G.2^-1*G.1^-1*G.2*G.2", "G.1*G.2*G.1^-1*G.2^-1*G.1^-1*G.2", 120)
("G.2*G.1^-1*G.2^-1*G.2^-1*G.1^-1*G.2*G.2", "G.2*G.1^-1*G.2^-1*G.1^-1*G.2*G.1", 120)
("G.1*G.2*G.1^-1*G.2*G.1*G.2^-1*G.2^-1", "G.1*G.2^-1*G.1^-1*G.2^-1*G.1*G.2", 120)
("G.1*G.2*G.1^-1*G.2*G.1*G.2^-1*G.2^-1", "G.2*G.1*G.2^-1*G.1^-1*G.2^-1*G.1", 120)
("G.1*G.2*G.1^-1*G.2*G.1*G.2^-1*G.2^-1", "G.1*G.2*G.1*G.2^-1*G.1^-1*G.2^-1", 120)
("G.1*G.2*G.1^-1*G.2*G.1*G.2^-1*G.2^-1", "G.2*G.1*G.2*G.1^-1*G.2^-1*G.1^-1", 120)
("G.1*G.2*G.1^-1*G.2*G.1*G.2^-1*G.2^-1", "G.1*G.2*G.1^-1*G.2^-1*G.1^-1*G.2", 120)
("G.1*G.2*G.1^-1*G.2*G.1*G.2^-1*G.2^-1", "G.2*G.1^-1*G.2^-1*G.1^-1*G.2*G.1", 120)
("G.2*G.1^-1*G.2^-1*G.1^-1*G.2*G.1*G.1", "G.2*G.1*G.2^-1*G.1*G.2*G.1^-1", 120)
("G.

("G.1*G.2^-1*G.1^-1*G.2*G.2*G.1^-1*G.2^-1", "G.1*G.2^-1*G.1^-1*G.2^-1*G.1*G.2", 120)
("G.1*G.2^-1*G.1^-1*G.2*G.2*G.1^-1*G.2^-1", "G.2*G.1*G.2^-1*G.1^-1*G.2^-1*G.1", 120)
("G.1*G.2^-1*G.1^-1*G.2*G.2*G.1^-1*G.2^-1", "G.1*G.2*G.1*G.2^-1*G.1^-1*G.2^-1", 120)
("G.1*G.2^-1*G.1^-1*G.2*G.2*G.1^-1*G.2^-1", "G.2*G.1*G.2*G.1^-1*G.2^-1*G.1^-1", 120)
("G.1*G.2^-1*G.1^-1*G.2*G.2*G.1^-1*G.2^-1", "G.1*G.2*G.1^-1*G.2^-1*G.1^-1*G.2", 120)
("G.1*G.2^-1*G.1^-1*G.2*G.2*G.1^-1*G.2^-1", "G.2*G.1^-1*G.2^-1*G.1^-1*G.2*G.1", 120)
("G.2*G.1*G.2^-1*G.1^-1*G.1^-1*G.2^-1*G.1", "G.2*G.1*G.2^-1*G.1*G.2*G.1^-1", 120)
("G.2*G.1*G.2^-1*G.1^-1*G.1^-1*G.2^-1*G.1", "G.1*G.2^-1*G.1^-1*G.2*G.1^-1*G.2^-1", 120)
("G.2*G.1*G.2^-1*G.1^-1*G.1^-1*G.2^-1*G.1", "G.2*G.1^-1*G.2*G.1*G.2^-1*G.1", 120)
("G.2*G.1*G.2^-1*G.1^-1*G.1^-1*G.2^-1*G.1", "G.1*G.2^-1*G.1*G.2*G.1^-1*G.2", 120)
("G.2*G.1*G.2^-1*G.1^-1*G.1^-1*G.2^-1*G.1", "G.2*G.1^-1*G.2^-1*G.1*G.2^-1*G.1^-1", 120)
("G.2*G.1*G.2^-1*G.1^-1*G.1^-1*G.2^-1*G.1", "G.1*G.2*G.1^-1*G.2*G.1*

...

("G.2*G.1*G.2^-1*G.2^-1*G.1*G.2*G.1^-1", "G.2*G.1^-1*G.2^-1*G.2^-1*G.1^-1*G.2*G.2", 120)
("G.2*G.1*G.2^-1*G.2^-1*G.1*G.2*G.1^-1", "G.2*G.2*G.1*G.2^-1*G.2^-1*G.2^-1*G.1", 120)
("G.2*G.1*G.2^-1*G.2^-1*G.1*G.2*G.1^-1", "G.2*G.1*G.2^-1*G.2^-1*G.2^-1*G.1*G.2", 120)
("G.2*G.1*G.2^-1*G.2^-1*G.1*G.2*G.1^-1", "G.1*G.2^-1*G.1^-1*G.2*G.1^-1*G.2^-1*G.1", 120)
("G.2*G.1*G.2^-1*G.2^-1*G.1*G.2*G.1^-1", "G.1*G.1*G.2^-1*G.1^-1*G.2*G.1^-1*G.2^-1", 120)
("G.2*G.1*G.2^-1*G.2^-1*G.1*G.2*G.1^-1", "G.2*G.1^-1*G.1^-1*G.2*G.1*G.2^-1*G.1", 120)
("G.2*G.1*G.2^-1*G.2^-1*G.1*G.2*G.1^-1", "G.2*G.2*G.2*G.1^-1*G.2^-1*G.2^-1*G.1^-1", 120)
("G.2*G.1*G.2^-1*G.2^-1*G.1*G.2*G.1^-1", "G.1*G.2^-1*G.1*G.2*G.1^-1*G.1^-1*G.2", 120)
("G.2*G.1*G.2^-1*G.2^-1*G.1*G.2*G.1^-1", "G.2*G.2*G.1^-1*G.2^-1*G.2^-1*G.1^-1*G.2", 120)
("G.2*G.1*G.2^-1*G.2^-1*G.1*G.2*G.1^-1", "G.2*G.1^-1*G.2^-1*G.1*G.1*G.2^-1*G.1^-1", 120)
("G.2*G.1*G.2^-1*G.2^-1*G.1*G.2*G.1^-1", "G.1*G.2*G.1^-1*G.1^-1*G.2*G.1*G.2^-1", 120)
("G.1*G.2*G.1*G.2^-1*G.1*G.2*G.1^-1"

("G.2*G.2*G.1*G.2^-1*G.1^-1*G.2^-1*G.1", "G.2*G.1*G.1*G.2*G.1^-1*G.2^-1*G.1^-1", 120)
("G.2*G.2*G.1*G.2^-1*G.1^-1*G.2^-1*G.1", "G.2*G.1^-1*G.2^-1*G.2^-1*G.2^-1*G.1^-1*G.2", 120)
("G.2*G.2*G.1*G.2^-1*G.1^-1*G.2^-1*G.1", "G.1*G.1*G.2*G.1^-1*G.2^-1*G.1^-1*G.2", 120)
("G.2*G.2*G.1*G.2^-1*G.1^-1*G.2^-1*G.1", "G.1*G.2*G.1^-1*G.2^-1*G.1^-1*G.2*G.1", 120)
("G.1*G.1*G.2*G.1^-1*G.1^-1*G.2*G.1^-1", "G.1*G.2*G.1*G.2^-1*G.2^-1*G.2^-1*G.2^-1", 120)
("G.1*G.1*G.2*G.1^-1*G.1^-1*G.2*G.1^-1", "G.2*G.1^-1*G.2^-1*G.1^-1*G.2*G.2*G.2", 120)
("G.1*G.1*G.2*G.1^-1*G.1^-1*G.2*G.1^-1", "G.2*G.2*G.1^-1*G.2^-1*G.1^-1*G.2*G.2", 120)
("G.1*G.1*G.2*G.1^-1*G.1^-1*G.2*G.1^-1", "G.2*G.1*G.2^-1*G.2^-1*G.2^-1*G.2^-1*G.1", 120)
("G.1*G.1*G.2*G.1^-1*G.1^-1*G.2*G.1^-1", "G.1*G.2^-1*G.2^-1*G.2^-1*G.2^-1*G.1*G.2", 120)
("G.1*G.1*G.2*G.1^-1*G.1^-1*G.2*G.1^-1", "G.2*G.2*G.2*G.2*G.1^-1*G.2^-1*G.1^-1", 120)
("G.1*G.1*G.2*G.1^-1*G.1^-1*G.2*G.1^-1", "G.2*G.2*G.2*G.1^-1*G.2^-1*G.1^-1*G.2", 120)
("G.2*G.2*G.1^-1*G.2^-1*G.2^-1*G.1^-1*G

("G.2*G.1^-1*G.2^-1*G.2^-1*G.2^-1*G.2^-1*G.1^-1", "G.2*G.1*G.2^-1*G.1^-1*G.1^-1*G.2^-1*G.2^-1", 120)
("G.2*G.1^-1*G.2^-1*G.2^-1*G.2^-1*G.2^-1*G.1^-1", "G.1*G.2*G.1*G.1*G.2*G.1^-1*G.1^-1", 120)
("G.2*G.1^-1*G.2^-1*G.2^-1*G.2^-1*G.2^-1*G.1^-1", "G.2*G.1^-1*G.1^-1*G.2^-1*G.1*G.2^-1*G.2^-1", 120)
("G.2*G.1^-1*G.2^-1*G.2^-1*G.2^-1*G.2^-1*G.1^-1", "G.2*G.2*G.1*G.1*G.2*G.1^-1*G.2^-1", 120)
("G.2*G.1^-1*G.2^-1*G.2^-1*G.2^-1*G.2^-1*G.1^-1", "G.1*G.1*G.1*G.2*G.1^-1*G.2*G.1^-1", 120)
("G.2*G.1^-1*G.2^-1*G.2^-1*G.2^-1*G.2^-1*G.1^-1", "G.2*G.1^-1*G.2^-1*G.1*G.2^-1*G.1^-1*G.2^-1", 120)
("G.2*G.1^-1*G.2^-1*G.2^-1*G.2^-1*G.2^-1*G.1^-1", "G.2*G.2*G.1^-1*G.2*G.1*G.1*G.2^-1", 120)
("G.2*G.1^-1*G.2^-1*G.2^-1*G.2^-1*G.2^-1*G.1^-1", "G.1*G.1*G.2^-1*G.1^-1*G.1^-1*G.2^-1*G.1^-1", 120)
("G.2*G.1^-1*G.2^-1*G.2^-1*G.2^-1*G.2^-1*G.1^-1", "G.2*G.1*G.2*G.1^-1*G.2*G.1*G.2^-1", 120)
("G.1*G.1*G.2*G.1^-1*G.1^-1*G.1^-1*G.2", "G.2*G.1*G.2^-1*G.1*G.2*G.1^-1*G.1^-1", 120)
("G.1*G.1*G.2*G.1^-1*G.1^-1*G.1^-1*G.2", "G.1*G.2^

("G.1*G.1*G.1*G.1*G.2*G.1^-1*G.2", "G.2*G.1^-1*G.2*G.1^-1*G.2^-1*G.2^-1*G.2^-1", 120)
("G.1*G.1*G.1*G.1*G.2*G.1^-1*G.2", "G.1*G.2^-1*G.2^-1*G.1^-1*G.2*G.1^-1*G.1^-1", 120)
("G.1*G.1*G.1*G.1*G.2*G.1^-1*G.2", "G.1*G.2*G.1^-1*G.2^-1*G.2^-1*G.1^-1*G.1^-1", 120)
("G.1*G.1*G.1*G.1*G.2*G.1^-1*G.2", "G.2*G.1*G.2*G.2*G.1*G.2^-1*G.2^-1", 120)
("G.1*G.1*G.1*G.1*G.2*G.1^-1*G.2", "G.1*G.1*G.2^-1*G.1*G.2*G.2*G.1^-1", 120)
("G.1*G.1*G.1*G.1*G.2*G.1^-1*G.2", "G.1*G.2*G.1*G.2^-1*G.1*G.2*G.1^-1", 120)
("G.1*G.1*G.1*G.1*G.2*G.1^-1*G.2", "G.2*G.2*G.1^-1*G.2^-1*G.2^-1*G.1^-1*G.2^-1", 120)
("G.1*G.1*G.1*G.1*G.2*G.1^-1*G.2", "G.1*G.2^-1*G.1^-1*G.2*G.1^-1*G.2^-1*G.1^-1", 120)
("G.1*G.1*G.1*G.1*G.2*G.1^-1*G.2", "G.2*G.2*G.2*G.1*G.2^-1*G.1*G.2^-1", 120)
("G.1*G.1*G.1*G.1*G.2*G.1^-1*G.2", "G.1*G.1*G.2*G.2*G.1*G.2^-1*G.1^-1", 120)
("G.2*G.2*G.2*G.2*G.1^-1*G.2^-1*G.1^-1", "G.1*G.2*G.1*G.2*G.1^-1*G.1^-1*G.1^-1", 120)
("G.2*G.2*G.2*G.2*G.1^-1*G.2^-1*G.1^-1", "G.2*G.1*G.1*G.2^-1*G.1^-1*G.2^-1*G.2^-1", 120)
("G.2*G.2*

("G.2*G.2*G.1^-1*G.2^-1*G.1*G.2^-1*G.1^-1", "G.2*G.1*G.2^-1*G.2^-1*G.2^-1*G.1*G.2", 120)
("G.2*G.2*G.1^-1*G.2^-1*G.1*G.2^-1*G.1^-1", "G.1*G.2^-1*G.1^-1*G.2*G.1^-1*G.2^-1*G.1", 120)
("G.2*G.2*G.1^-1*G.2^-1*G.1*G.2^-1*G.1^-1", "G.1*G.1*G.2^-1*G.1^-1*G.2*G.1^-1*G.2^-1", 120)
("G.2*G.2*G.1^-1*G.2^-1*G.1*G.2^-1*G.1^-1", "G.2*G.1^-1*G.1^-1*G.2*G.1*G.2^-1*G.1", 120)
("G.2*G.2*G.1^-1*G.2^-1*G.1*G.2^-1*G.1^-1", "G.2*G.2*G.2*G.1^-1*G.2^-1*G.2^-1*G.1^-1", 120)
("G.2*G.2*G.1^-1*G.2^-1*G.1*G.2^-1*G.1^-1", "G.1*G.2^-1*G.1*G.2*G.1^-1*G.1^-1*G.2", 120)
("G.2*G.2*G.1^-1*G.2^-1*G.1*G.2^-1*G.1^-1", "G.2*G.2*G.1^-1*G.2^-1*G.2^-1*G.1^-1*G.2", 120)
("G.2*G.2*G.1^-1*G.2^-1*G.1*G.2^-1*G.1^-1", "G.2*G.1^-1*G.2^-1*G.1*G.1*G.2^-1*G.1^-1", 120)
("G.2*G.2*G.1^-1*G.2^-1*G.1*G.2^-1*G.1^-1", "G.1*G.2*G.1^-1*G.1^-1*G.2*G.1*G.2^-1", 120)
("G.1*G.1*G.2*G.2*G.1*G.2^-1*G.1^-1", "G.2*G.1^-1*G.2*G.1*G.1*G.1*G.1", 120)
("G.1*G.1*G.2*G.2*G.1*G.2^-1*G.1^-1", "G.1*G.2*G.1^-1*G.2*G.1*G.1*G.1", 120)
("G.1*G.1*G.2*G.2*G.1*G.2^-1*G

In [ ]:
Сохранение результатов:

In [7]:
file = open("output.txt", "w")

for el in result
    write(file, "$el\n")
end